<a href="https://colab.research.google.com/github/Blacksmith1111/Optics/blob/main/HomeTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

class MyModulator:
  def __init__(self, bits_number, mod_type = "QPSK", snr = 10):
    self.mod_type = mod_type
    #self.bits_stream = np.random.binomial(n = 1, p = 0.5, size = bits_number)
    self.bits_stream = [1,1,1,0,1,0,1,0,0,0,0,0]
    self.snr = snr

  def bits_to_symbols(self):
    self.symbols = []
    if self.mod_type == "16QAM":
      for i in range(0, len(self.bits_stream), 4):
        if i + 4 <= len(self.bits_stream):
            self.symbols.append(self.bits_stream[i:i+4])
    elif self.mod_type == "QPSK":
      for i in range(0, len(self.bits_stream), 2):
        if i + 2 <= len(self.bits_stream):
          self.symbols.append(self.bits_stream[i:i+2])
    self.symbols = np.array(self.symbols)
    return self.symbols

  def mapper(self):
    self.I = np.zeros(len(self.symbols))
    self.Q = np.zeros(len(self.symbols))
    self.bin_vals = np.zeros(self.symbols.shape)
    if self.mod_type == "16QAM":
      Levels = np.array([1, 3])
      for index, symbol in enumerate(self.symbols):
        self.bin_vals[index] = symbol
        self.I[index] = Levels[0] if (str(symbol[1]) + str(symbol[3]) == "11" or  str(symbol[1]) + str(symbol[3]) == "10") else Levels[1]
        self.I[index] *= -1 if str(symbol[0]) + str(symbol[2]) == "01" or str(symbol[0]) + str(symbol[2]) == "00" else 1
        self.Q[index] = Levels[0] if str(symbol[1]) + str(symbol[3]) == "11" or str(symbol[1]) + str(symbol[3]) == "01" else Levels[1]
        self.Q[index] *= -1 if str(symbol[0]) + str(symbol[2]) == "00" or str(symbol[0]) + str(symbol[2]) == "10" else 1
    elif self.mod_type == "QPSK":
      Levels = np.array([-1, 1])
      for index, symbol in enumerate(self.symbols):
        self.bin_vals[index] = symbol
        self.I[index] = Levels[0] if symbol[0] == 0 else Levels[1]
        self.Q[index] = Levels[0] if symbol[1] == 0 else Levels[1]

    self.mapped = self.I + 1j*self.Q
    return self.mapped

  def Noise_generator(self):
    points_amount = len(self.mapped)
    root_points_power = np.linalg.norm(self.mapped) / np.sqrt(points_amount)
    # print(10 * np.log10(root_points_power ** 2))
    print(root_points_power)
    noise_power = root_points_power / (2 * (10 ** (self.snr / 10)))
    real_part = np.random.normal(scale = np.sqrt(noise_power), size = points_amount)
    imag_part = np.random.normal(scale = np.sqrt(noise_power), size = points_amount)
    return real_part + 1j * imag_part

  def plot_constellation(self):
    plt.figure(figsize=(8, 8))
    plt.scatter(np.real(self.mapped), np.imag(self.mapped), color='red')
    for (x, y, txt) in zip(np.real(self.mapped), np.imag(self.mapped), self.bin_vals):
      plt.text(x, y, str(txt), fontsize=13)

    if self.mod_type == "16QAM":
      plt.title('16QAM Constellation')
      plt.xlim(-4, 4)
      plt.ylim(-4, 4)
    elif self.mod_type == "QPSK":
      plt.title('QPSK Constellation')
      plt.xlim(-4, 4)
      plt.ylim(-4, 4)

    plt.xlabel('I')
    plt.ylabel('Q')
    plt.grid(True)
    plt.axhline(0, color='black', lw=0.5)
    plt.axvline(0, color='black', lw=0.5)
    plt.show()





In [35]:
class MyDemodulator:
  def __init__(self, mapped_symbols, mod_type = "QPSK"):
    self.mod_type = mod_type
    self.mapped = mapped_symbols
    if self.mod_type == "16QAM":
      self.symbol = np.zeros(4)
    elif self.mod_type == "QPSK":
      self.symbol = np.zeros(2)

  def demodulate(self):
    out_symbol = []
    self.out_bits = []
    ref_arr = []
    if self.mod_type == "16QAM":
      dict_16 = {-3 + 3j: "1000", -3 + 1j: "0011", -3 - 3j: "0000", -3 - 1j: "0001",
                 -1 + 3j: "0110", -1 + 1j: "0111", -1 - 3j: "0100", -1 - 1j: "0101",
                  3 + 3j: "1010",  3 + 1j: "1011",  3 - 3j: "1000",  3 - 1j: "1001",
                  1 + 3j: "1110",  1 + 1j: "1111",  1 - 3j: "1100",  1 - 1j: "1101"}
      levels = np.array([-3, -1, 1, 3])

      for i in range(len(levels)):
        for k in range(len(levels)):
          ref_arr.append(levels[i] + 1j * levels[k])
      ref_arr = np.array(ref_arr)
      norms = np.zeros(len(ref_arr))
      for index in range(len(self.mapped)):
        for sec_ind in range(len(ref_arr)):
          norms[sec_ind] = np.linalg.norm(ref_arr[sec_ind] - self.mapped[index])
        out_symbol.append(ref_arr[np.argmin(norms)])
      out_symbol = np.array(out_symbol)
      for index in range(len(self.mapped)):
        self.out_bits.append(dict_16[out_symbol[index]])
      self.out_bits = np.array([int(digit) for string in self.out_bits for digit in string])

    elif self.mod_type == "QPSK":
      dict_qpsk = {1 + 1j: "11", -1 +1j: "01", -1 -1j: "00", 1 - 1j: "10"}
      levels = np.array([-1, 1])
      for i in range(len(levels)):
        for k in range(len(levels)):
          ref_arr.append(levels[i] + 1j * levels[k])
      ref_arr = np.array(ref_arr)
      norms = np.zeros(len(ref_arr))
      for index in range(len(self.mapped)):
        for sec_ind in range(len(ref_arr)):
          norms[sec_ind] = np.linalg.norm(ref_arr[sec_ind] - self.mapped[index])
        out_symbol.append(ref_arr[np.argmin(norms)])
      out_symbol = np.array(out_symbol)
      for index in range(len(self.mapped)):
        self.out_bits.append(dict_qpsk[out_symbol[index]])
      self.out_bits = np.array([int(digit) for string in self.out_bits for digit in string])



    return self.out_bits


In [43]:
num_bits = 12
###test 16QAM
obj = MyModulator(num_bits, "16QAM")
print(obj.bits_to_symbols(), obj.bits_to_symbols().shape, "\n")
print(obj.mapper(), "\n")
#obj.plot_constellation()
###test demodulator
dem = MyDemodulator(obj.mapped, "16QAM")
print(dem.demodulate(), type(dem.demodulate()[0]), "\n")
###test QPSK

#newobj.plot_constellation()
### with noise
no = MyModulator(num_bits, "16QAM", 20)
no.bits_to_symbols()
print(no.mapper() + no.Noise_generator(), "\n")
yes = MyDemodulator(no.mapper() + no.Noise_generator(), "16QAM")
print(yes.demodulate())

[[1 1 1 0]
 [1 0 1 0]
 [0 0 0 0]] (3, 4) 

[ 1.+3.j  3.+3.j -3.-3.j] 

[1 1 1 0 1 0 1 0 0 0 0 0] <class 'numpy.int64'> 

3.915780041490244
[ 1.25221688+4.70736848j -3.44821935+7.57870838j -1.43814408+4.99110529j] 

3.915780041490244
[0 0 1 1 1 0 1 1 0 0 1 1]
